<a href="https://colab.research.google.com/github/dangdang2222/LG_Aimers/blob/main/nn_%EB%B6%84%EB%A5%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
train = pd.read_csv('./drive/MyDrive/LG_Aimer/train.csv')
test = pd.read_csv('./drive/MyDrive/LG_Aimer/test.csv')
meta_x_features= pd.read_csv('./drive/MyDrive/LG_Aimer/meta/x_feature_info.csv')
meta_y_features= pd.read_csv('./drive/MyDrive/LG_Aimer/meta/y_feature_info.csv')

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
import random
import os
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED']=str(seed)
    np.random.seed(seed)
seed_everything(42)

train_df = pd.read_csv('./drive/MyDrive/LG_Aimer/train.csv')
drop_sensors = ['ID','X_04', 'X_23', 'X_47', 'X_48']
train_df= train_df.drop(labels = drop_sensors,axis=1, inplace=False)
train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

In [ ]:
meta_x_features

In [6]:
df = pd.read_csv('./drive/MyDrive/LG_Aimer/train.csv')
df1 = df.copy()
df1=df[(df['Y_01']>=0.2)&(df['Y_01']<2.0)&(df['Y_02']>=0.2)
&(df['Y_02']<=2.1)&(df['Y_02']>=0.2)
&(df['Y_03']>=0.2)&(df['Y_03']<=2.1)
&(df['Y_04']<=19)&(df['Y_04']>=7)&(df['Y_05']<=36.5)&(df['Y_05']>=22)
&(df['Y_06']>=-19.2)&(df['Y_06']<=19)&(df['Y_07']>=2.4)&(df['Y_07']<=4)
&(df['Y_08']>=-29.2)&(df['Y_08']<=-24)&(df['Y_09']>=-29.2)&(df['Y_09']<=-24)
&(df['Y_10']>=-30.6)&(df['Y_10']<=-20)&(df['Y_11']>=19.6)&(df['Y_11']<=26.6)
&(df['Y_12']>=-29.2)&(df['Y_12']<=-24)&(df['Y_13']>=-29.2)&(df['Y_13']<=-24)
&(df['Y_14']>=-29.2)&(df['Y_14']<=-24)]

for idx in range(0,39607):
    if idx in df1.index:
        df.loc[idx,'pass']=1
    else:
        df.loc[idx,'pass']=0

In [7]:
from sklearn.model_selection import train_test_split

drop_sensors = ['ID','X_04', 'X_23', 'X_47', 'X_48','X_14','X_15','X_16','X_17','X_18','Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06', 'Y_07', 'Y_08', 'Y_09',
       'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14']
df= df.drop(labels = drop_sensors,axis=1, inplace=False)
label = df['pass']
df= df.drop(labels = 'pass',axis=1, inplace=False)
print(len(df.columns))
x_train, x_test, y_train, y_test = train_test_split(df, label, test_size=0.3, random_state=10)

47


In [20]:
x_train = x_train.to_numpy().reshape(x_train.shape[0], x_train.shape[1],1)
print(x_train.shape)
x_test = x_test.to_numpy().reshape(x_test.shape[0],x_test.shape[1],1)
print(x_test.shape)

(27724, 47, 1)
(11883, 47, 1)


## DNN 분류기

In [21]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input, Bidirectional,TimeDistributed, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.callbacks import EarlyStopping
from sklearn.model_selection import RepeatedKFold
from tensorflow.keras import optimizers

model = Sequential()
model.add(Input(shape=(47,1)))
model.add(Dense(256,activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 47, 256)           512       
                                                                 
 dense_7 (Dense)             (None, 47, 128)           32896     
                                                                 
 dense_8 (Dense)             (None, 47, 64)            8256      
                                                                 
 flatten_1 (Flatten)         (None, 3008)              0         
                                                                 
 dense_9 (Dense)             (None, 128)               385152    
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dense_11 (Dense)            (None, 1)                

In [ ]:
early_stopping = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 30, verbose = 1)
hist = model.fit(x_train, y_train, epochs = 1000, batch_size = 20, validation_data = (x_test, y_test),callbacks = [early_stopping])

In [27]:
preds = model.predict(x_test)

In [57]:
predss = []

count = 0 
for i in preds:
  if i < 0.907: #임계값 바꾸기
    count+=1
    predss.append(0)
  else:
    predss.append(1)
print(count)

1162


In [56]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, predss)
print(cm)

[[   78  1081]
 [  594 10130]]
